In [ ]:
import os

host = os.getenv('API_HOST')
port = os.getenv('API_PORT')

In [ ]:
print host, port

In [ ]:
client_id = os.getenv('CLIENT_ID')
user = os.getenv('XT_USER')
password = os.getenv('XT_PASSWORD')

In [ ]:
base_endpoint = 'http://{}:{}'.format(host, port)
print base_endpoint

In [ ]:
# Login
import requests

login_payload = {}
login_payload['client_id'] = client_id
login_payload['username'] = user
login_payload['password'] = password

print login_payload

#login_payload = {"client_id": "apiAppIdForTesting", "username": "xapi-01", "password":"Tester01"}
headers = {"Content-Type":"application/json"}

In [ ]:
import json
login_endpoint = base_endpoint + '/xtrac/login'
print 'sending request to {}'.format(login_endpoint)
r = requests.post(login_endpoint, data=json.dumps(login_payload))

In [ ]:
print r.raise_for_status()

In [ ]:
print r.text

In [ ]:
login_response = json.loads(r.text)
print login_response['access_token']

In [ ]:
headers['Authorization'] = 'Bearer {}'.format(login_response['access_token'])
print headers

In [ ]:
create_work_item = '''
{
  "itemType": "Miscellaneous",
  "status": "Assign Project Mgr",
  "parties": [
    {
      "name": "ORIG",
      "fields": [
        {
          "name": "OrgName",
          "values": [
            "test"
          ]
        }
      ]
    }
  ]
}
'''

In [ ]:
create_url = base_endpoint + '/xtrac/api/v1/work-items?evaluateRule=true'
print 'sending request to {}'.format(create_url)
r = requests.post(create_url, data=create_work_item, headers=headers)

In [ ]:
print r.raise_for_status()

In [ ]:
print r.text

In [ ]:
create_response = json.loads(r.text)
work_item_no = create_response['data']['workItemNumber']
print 'created {}'.format(work_item_no)

In [ ]:
def update_work_item(workItemNo, update, headers):
    update_url = '{}/xtrac/api/v1/work-items/{}?evaluateRule=true'.format(
        base_endpoint, workItemNo
    )

    r = requests.put(update_url,data=update,headers=headers)
    r.raise_for_status()
    print r.status_code
    return r.text

In [ ]:
# Status -> Review Tech Requirements
update_response = update_work_item(work_item_no, '{"status":"Review Tech Requirements"}', headers)
print update_response

In [ ]:
# Status -> Configure Solution
update_response = update_work_item(work_item_no, '{"status":"Configure Solution"}', headers)
print update_response

In [ ]:
notify_email = 'foo@bar'
email_update = '''
{
    "status":"Send Email",
    "fields": [
    {
        "name": "Memo",
        "values": [
            "''' + notify_email + '''"
        ],
        "isMsf": false
    }
    ]
}
'''

In [ ]:
# Status -> Send Email
update_response = update_work_item(work_item_no, email_update, headers)
print update_response

In [ ]:
last_update = '''
{
    "status":"Task Complete",
    "fields": [
    {
        "name": "Memo",
        "values": [
            ""
        ],
        "isMsf": false
    }
    ]
}
'''

In [ ]:
# Status -> Task Complete
update_response = update_work_item(work_item_no, last_update, headers)
print update_response